# This is just a notebook to visualise 1kHz filtered raw data

## Setup everything

### Import packages

In [3]:
import os
import sys
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
from open_ephys.analysis import Session

import mmap
import xarray as xr
import dask.array as da
#%matplotlib widget

from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter


from ipyfilechooser import FileChooser
import ipywidgets as widgets
import pickle
import configparser

### Import local config

In [103]:
configFN = 'localConfig.ini'
config = configparser.ConfigParser()
config.read(configFN)

rawDataPath = config['DATA']['path']

def updateConf():
    with open(configFN, 'w') as configfile:
        config.write(configfile)

### Define functions to deal with config dictionaries

channelsMap is a dictionary with one key per brain region and/or canal of interest (TTL). The value associated to the key is an array with every corresponding canal. The "status" key defines which of these canals to use for analyses:
- 0: not to use
- 1 only: floating point canal
- 1 and 2: differential signal 2-1

In [ ]:
def getPathComponent(filename,projectType):
    
    dirPathComponents = os.path.normpath(filename).split(os.sep)
    expeInfo = dict()

    expeInfo['analysisPath'] = os.path.join('/',*dirPathComponents[0:-5])
    expeInfo['ProjectID'] = dirPathComponents[-5]
    expeInfo['subProjectID'] = dirPathComponents[-4]

    with open(os.path.join('/',*dirPathComponents[0:-3],'projectConfig.pkl'), 'rb') as f:
        loaded_dict = pickle.load(f)
        expeInfo['projectType'] = loaded_dict['projectType']

    if projectType == 0:
        expeInfo['conditionID'] = dirPathComponents[-3]
        expeInfo['AnimalID'] = dirPathComponents[-2]
    else:
        expeInfo['AnimalID'] = dirPathComponents[-3]
        expeInfo['conditionID'] = dirPathComponents[-2]
        
    expeInfo['recordingID'] = dirPathComponents[-1]

    return expeInfo


In [106]:
expeInfo = dict()

projects = [p.split('.')[0] for p in config.sections() if p not in ['DATA','ANALYSIS']]

subprojects = {p.split('.')[0]: widgets.Dropdown(
    options=[p.split('.')[1]],
    description='Sub-project:',
    ) for p in config.sections() if p not in ['DATA','ANALYSIS']} 


def printExpeInfo(**func_kwargs):
    pass#print(expeInfo)

def updateProject(widget):
    ProjectID = widget.new
    expeInfo['ProjectID'] = ProjectID
    new_i = widgets.interactive(printExpeInfo, project=wProject, subProject=subprojects[ProjectID])
    i.children = new_i.children


def updateSubProject(widget):
    if widget['type'] == 'change' and widget['name'] == 'value':
        expeInfo['subProjectID'] = widget.new

def update_design(widget):
    projectType = widget.new
    if projectType == 0:
        new_i = widgets.interactive(printExpeInfo, project=wProject, subProject=subprojects[ProjectID], design=wDesign, condition=wCondition, animal=wAnimal, rec=wRec)
    else:
        new_i = widgets.interactive(printExpeInfo, project=wProject, subProject=subprojects[ProjectID], design=wDesign, animal=wAnimal, condition=wCondition, rec=wRec)
    i.children = new_i.children
    %store projectType

analysisPath = config['ANALYSIS']['path']
projectType = int(config['ANALYSIS']['projectType'])
ProjectID = config['ANALYSIS']['ProjectID']
subProjectID = config['ANALYSIS']['subProjectID']
conditionID = config['ANALYSIS']['conditionID']
AnimalID = int(config['ANALYSIS']['AnimalID'])
recordingID = int(config['ANALYSIS']['recordingID'])

wProject = widgets.Dropdown(
    options=projects,
    value=ProjectID,
    description='Project:',
    disabled=False,
)
wProject.observe(updateProject, 'value')

#wSubProject.observe(updateSubProject)

designs = ['independant groups', 'within subject']
wDesign = widgets.RadioButtons(
    options=designs,
    value=designs[projectType], # Defaults to 'independant groups'
    description='Experiment design:'
)
wDesign.observe(update_design, names=['index'])

wAnimal = widgets.BoundedIntText(
    value=AnimalID,
    min=0,
    #max=10,
    step=1,
    description='Animal ID:'
)

conditions = ["bla", "ctr"]
wCondition = widgets.Dropdown(
    options=conditions,
    value=conditionID,
    description='Condition:',
)

wRec = widgets.BoundedIntText(
    value=recordingID,
    min=0,
    #max=10,
    step=1,
    description='Recording ID:'
)

if projectType == 0:
    i = widgets.interactive(printExpeInfo, project=wProject, subProject=subprojects[ProjectID], design=wDesign, condition=wCondition, animal=wAnimal, rec=wRec)
else:
    i = widgets.interactive(printExpeInfo, project=wProject, subProject=subprojects[ProjectID], design=wDesign, animal=wAnimal, condition=wCondition, rec=wRec)


def defineExpeInfo():
    display(i)

In [3]:
def generateConfigDict(filename):
    
    numChanels=64

    channelsMap = dict( \
        EMG = [dict(canal = 6, status=1)],
        PFC = [dict(canal = 5, status=1),
            dict(canal = 4, status=2)
            ],
        CA1 = [dict(canal = 8, status=1),
            dict(canal = 0, status=0),
            dict(canal = 1, status=0),
            ],
        TTL = [dict(canal = 10, status=1)],
    )
    
    rawDataPath = None

    projectType = int(config['ANALYSIS']['projectType'])
    expeInfo = getPathComponent(filename,projectType)

    allParamsDict = dict(channelsMap, numChanels = numChanels, rawDataPath = rawDataPath, expeInfo = expeInfo)

    with open(filename, 'wb') as f:
        pickle.dump(allParamsDict, f)
            

In [4]:
def loadConfigDict(filename):
    
    with open(filename, 'rb') as f:
        loaded_dict = pickle.load(f)

        numChanels = loaded_dict.pop('numChanels')
        print(numChanels)

        rawDataPath = loaded_dict.pop('rawDataPath')
        print(rawDataPath)

        expeInfo = loaded_dict.pop('expeInfo')
        print(expeInfo)

        channelsMap = loaded_dict
        print(channelsMap)

### Choose experiment
Select the experiment to display. If the experiment was already analyzed, a saved_dictionary.pkl was created and contains all necessary variables. Select this file. Otherwise select the raw data recording file.

In [108]:
currentFile = None
%store -r currentFile

if currentFile is not None: # a file is currently being used
    pathName, fileName = os.path.split()
else:
    pathName = rawDataPath
    fileName = ""
     
fc = FileChooser(path=pathName, filename=fileName, select_default=True, show_only_dirs = False, title = "<b>Select file</b>")
display(fc)

def update_my_folder(chooser):
    selection = chooser.selected
    if selection.endswith("pkl"):
        currentFile = selection
        loadConfigDict(selection)
        %store currentFile
    else:
        print("this is not a config file and we should deal with that")
        defineExpeInfo()
    
# Register callback function
fc.register_callback(update_my_folder)

no stored variable or alias currentFile


FileChooser(path='/Users/mb/Documents/Syntuitio/AudreyHay/PlanB', filename='', title='<b>Select file</b>', sho…

In [6]:





dictName = 'saved_dictionary.pkl'

if ProjectID == "":
    ProjectID = input('No project defined, please give it a name:')
    config.set('ANALYSIS', 'ProjectID', ProjectID)
    updateConf()
    print("The project ID is set to : {}".format(ProjectID))

if subProjectID == "":
    subProjectID = input('No sub project defined, please give it a name:')
    config.set('ANALYSIS', 'subProjectID', subProjectID)
    updateConf()
    print("The project ID is set to : {}".format(ProjectID))

    print("The sub-project ID is not in strore yet, setting it to default: {}".format(subProjectID))



designs = ['independant groups', 'within subject']
w = widgets.RadioButtons(
    options=designs,
    value=designs[projectType], # Defaults to 'independant groups'
    # layout={'width': 'max-content'}, # If the items' names are long
    description='Experiment design:',
    disabled=False
)
display(w)

def update_design(widget):
    projectType = widget.new
    %store projectType

w.observe(update_design, names=['index'])

try:
    if projectType == 0:
        path = os.path.join(analysisPath, ProjectID, subProjectID, conditionID, str(AnimalID), str(recordingID))
        
    else:
        path = os.path.join(analysisPath, ProjectID, subProjectID, str(AnimalID), conditionID, str(recordingID))
    os.makedirs(path, exist_ok=True)
    fc = FileChooser(path=path, filename=dictName, select_default=True, show_only_dirs = True, title = "<b>Analysis path</b>")
except Exception as error:
    print(error)
    fc = FileChooser(filename=dictName, show_only_dirs = True, title = "<b>Analysis path</b>")
display(fc)

# Sample callback function
def update_my_folder(chooser):
    global analysisPath, projectType, AnimalID, ProjectID, subProjectID, conditionID, recordingID
    dirPath = os.path.normpath(chooser.selected)
    dirPathComponents = dirPath.split(os.sep)
    analysisPath = os.path.join('/',*dirPathComponents[0:-5])
    ProjectID = dirPathComponents[-5]
    subProjectID = dirPathComponents[-4]
    if projectType == 0:
        conditionID = dirPathComponents[-3]
        AnimalID = dirPathComponents[-2]
    else:
        AnimalID = dirPathComponents[-3]
        conditionID = dirPathComponents[-2]
    recordingID = dirPathComponents[-1]
    print("Storing analysis path: {}".format(analysisPath))
    print("Storing project: {}".format(ProjectID))
    print("Storing sub-project: {}".format(subProjectID))
    print("Storing condition: {}".format(conditionID))
    print("Storing animal ID: {}".format(AnimalID))
    print("Storing recording ID: {}".format(recordingID))
    %store analysisPath ProjectID subProjectID conditionID AnimalID recordingID

    config.set('ANALYSIS', 'ProjectID', ProjectID)

    with open(configFN, 'w') as configfile:
        config.write(configfile)

    return 

# Register callback function
fc.register_callback(update_my_folder)

RadioButtons(description='Experiment design:', options=('independant groups', 'within subject'), value='indepe…

FileChooser(path='/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/Analysis/Try/None/None/1/1', filename='saved_d…

### Get config file if it exists, generate it otherwise

In [7]:
if projectType == 0:
    path = os.path.join(analysisPath, ProjectID, subProjectID, conditionID, str(AnimalID))
else:
    path = os.path.join(analysisPath, ProjectID, subProjectID, str(AnimalID))

if not os.path.isfile(os.path.join(path,dictName)):
    generateConfigDict(os.path.join(path,dictName))

with open(os.path.join(path,dictName), 'rb') as f:
    loaded_dict = pickle.load(f)

    numChanels = loaded_dict.pop('numChanels')
    print(numChanels)

    rawDataPath = loaded_dict.pop('rawDataPath')
    print(rawDataPath)

    channelsMap = loaded_dict
    print(channelsMap)
    

64
None
{'EMG': [{'canal': 6, 'status': 1}], 'PFC': [{'canal': 5, 'status': 1}, {'canal': 4, 'status': 2}], 'CA1': [{'canal': 8, 'status': 1}, {'canal': 0, 'status': 0}, {'canal': 1, 'status': 0}], 'TTL': [{'canal': 10, 'status': 1}]}


#### experiment path
Select the experiment to analyse. If variables are already defined for that particular experiment, they will be used. Otherwise you will have to define them in the next section.

In [8]:
if rawDataPath is not None:
    dirPathComponents = rawDataPath.split(os.sep)
    expePath = os.path.join('/',*dirPathComponents[:-4])
    subpath4FPGA = os.path.join(*dirPathComponents[-4:-1])
    fileBaseName = dirPathComponents[-1]
else:
    expePath = None
    subpath4FPGA = None
    fileBaseName = None
    #%store -r expePath subpath4FPGA fileBaseName
        
    if expePath is None:
        expePath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/"
        print("The experiment path is not in strore yet, setting it to default: {}".format(expePath))

    if subpath4FPGA is None:
        subpath4FPGA = "recording8/continuous/Rhythm_FPGA-133.0/"
        print("The experiment path is not in strore yet, setting it to default: {}".format(subpath4FPGA))

    if fileBaseName is None:
        fileBaseName = "continuous.dat"
        print("The experiment path is not in strore yet, setting it to default: {}".format(fileBaseName))


try:
    fc = FileChooser(path=os.path.join(expePath,subpath4FPGA), filename=fileBaseName,select_default=True, show_only_dirs = False, title = "<b>ePhys data</b>")
except:
    fc = FileChooser(show_only_dirs = False, title = "<b>ePhys data</b>")
display(fc)

# Sample callback function
def update_my_folder(chooser):
    global expePath, subpath4FPGA, fileBaseName
    filePath = chooser.selected
    subpath4FPGA = ""
    fileBaseName = os.path.basename(filePath)
    print("Storing filename: {}".format(fileBaseName))
    level=4
    for i in range(level):
        filePath = os.path.dirname(filePath)
        if i < level-1:
            subpath4FPGA = os.path.basename(filePath) + "/" + subpath4FPGA
    expePath = filePath
    print("Storing experiement path: {}".format(expePath))
    print("Storing subpath: {}".format(subpath4FPGA))

    with open(os.path.join(path,dictName), 'a+') as f:
        pickle.dump(rawDataPath,f)

    %store expePath subpath4FPGA fileBaseName
    return 

# Register callback function
fc.register_callback(update_my_folder)

The experiment path is not in strore yet, setting it to default: /Users/mb/Documents/Syntuitio/AudreyHay/PlanB/
The experiment path is not in strore yet, setting it to default: recording8/continuous/Rhythm_FPGA-133.0/
The experiment path is not in strore yet, setting it to default: continuous.dat


FileChooser(path='/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/recording8/continuous/Rhythm_FPGA-133.0', file…

## Load Data

### Map the whole data into memory

In [9]:
filename = os.path.join(expePath,subpath4FPGA,fileBaseName)

if fileBaseName == "continuous.dat":
    All = np.memmap(filename, mode='r', dtype='int16')
    #All = np.fromfile(filename, dtype="int16")
    All = All.reshape(-1,numChanels)
elif fileBaseName.endswith(".npy"):
    All = np.load(filename, mmap_mode= 'r')

In [10]:
filenameT = filename = os.path.join(expePath,subpath4FPGA,"timestamps.npy")
Timestamps = np.load(filenameT)
Timestamps.shape
Timestamps = Timestamps*2000
Timestamps = Timestamps.astype(int)
Timestamps

array([506597888000, 506597890000, 506597892000, ..., 553625594000,
       553625596000, 553625598000])

### Extract submatrix of interest

In [11]:
start = 000000
end = All.shape[0]
combined = np.empty((end-start,0),np.int16)
channelLabels = []
for region in channelsMap:
    print(region, "->", channelsMap[region])
    if len([canal["canal"] for canal in channelsMap[region] if canal["status"]==2])>0:
        c2 = [canal["canal"] for canal in channelsMap[region] if canal["status"]==2][0]
        c1 = [canal["canal"] for canal in channelsMap[region] if canal["status"]==1][0]
        print("Getting differential signal of channel {} - channel {} for {}".format(c2,c1,region))
        channelLabels.append(region)
        combined = np.append(combined, All[start:end, c2, np.newaxis] - All[:, c1, np.newaxis], axis=1)
    elif len([canal["canal"] for canal in channelsMap[region] if canal["status"]==1])>0:
        c = [canal["canal"] for canal in channelsMap[region] if canal["status"]==1][0]
        print("Getting floating signal of channel {} for {}".format(c,region))
        combined = np.append(combined, All[start:end,c, np.newaxis], axis=1)
        channelLabels.append(region)

EMG -> [{'canal': 6, 'status': 1}]
Getting floating signal of channel 6 for EMG
PFC -> [{'canal': 5, 'status': 1}, {'canal': 4, 'status': 2}]
Getting differential signal of channel 4 - channel 5 for PFC
CA1 -> [{'canal': 8, 'status': 1}, {'canal': 0, 'status': 0}, {'canal': 1, 'status': 0}]
Getting floating signal of channel 8 for CA1
TTL -> [{'canal': 10, 'status': 1}]
Getting floating signal of channel 10 for TTL


In [12]:
%gui qt 
# allows the app to be closed clean and reopen
app = mkQApp()


sample_rate = 1000.
t_start = 0.

TTL = Timestamps

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: TTL, 1: TTL}
#and asign them to some channels each
scatter_channels = {0: [0, 12], 1: [0, 1]}
source = AnalogSignalSourceWithScatter(All, sample_rate, t_start, scatter_indexes, scatter_channels)


#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(combined, sample_rate, t_start, 'Signals', channel_names=channelLabels)
#view1 = TraceViewer(source=source)
win.add_view(view1)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True

view1.by_channel_params['ch0', 'gain'] = 0.00002
view1.by_channel_params['ch1', 'gain'] = 0.00004
view1.by_channel_params['ch2', 'gain'] = 0.00004
view1.by_channel_params['ch3', 'gain'] = 0.00005
#view1.by_channel_params['ch4', 'gain'] = 0.00005

view1.by_channel_params['ch0', 'offset'] = 0.1
view1.by_channel_params['ch1', 'offset'] = 0.05
view1.by_channel_params['ch2', 'offset'] = -0.0
view1.by_channel_params['ch3', 'offset'] = -0.05
#view1.by_channel_params['ch4', 'offset'] = -0.1

#And also parameters for each channel
view1.by_channel_params['ch0', 'color'] = '#aa0000'
view1.by_channel_params['ch1', 'color'] = '#ff5500'
view1.by_channel_params['ch2', 'color'] = '#9b3b70'
view1.by_channel_params['ch3', 'color'] = '#0055ff'


#Run
win.show()
app.exec()

0

: 